# 1. GAN 소개 / Introduction to GAN
### Contents
 - 개요
 - 1.1 개념
   - (1.1.1 원리)
   - (1.1.2 종류)
 - 1.2 GAN 모델
   - 1.2.1 모델 구조
   - 1.2.2 CGAN
   - 1.2.3 평가메트릭  
 - 1.3 적용사례
   - 1.3.1 가짜이미지 / NVIDIA
   - 1.3.2 오바마 가짜연설영상 / 워싱턴대학교
   - 1.3.3 Eye In-Painting / Facebook
 - 1.4 한계점
   
<hr>

# 1. GAN 소개 (Introduction to GAN)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pseudo-Lab/Tutorial-Book/blob/master/book/chapters/GAN/Ch1-Introductions.ipynb)

## 개요 (Overview)

GAN은 Generative Adversarial Nets의 약자로 우리말로는 "적대적 생성 신경망"이라고 번역되는 AI기술 중 하나입니다. 이름에서 알 수 있듯 GAN은 두 개의 네트워크를 적대적으로(adversarial) 학습시키며 실제 데이터와 비슷한 데이터를 생성(generative)해내는 모델인데 이렇듯 정해진 label값이 없기 때문에 비지도 학습 기반 생성모델로 분류됩니다. 

GAN은 구글 브레인에서 머신러닝을 연구했던 Ian Goodfellow에 의해 2014년 처음으로 신경정보처리시스템학회(NIPS)에서 제안되었고, 이후 텍스트 생성, 이미지 생성 등에 다양하게 응용되고 있습니다.

## 1.1. 개념 (Concepts)

GAN은 Generator(G,생성모델/생성기)와 Discriminator(D,판별모델/판별기) 2개의 서로 다른 네트워크로 이루어져 있으며 이 두 네트워크를 적대적으로 학습시키며 목적을 달성합니다. 생성모델(G)의 목적은 진짜 분포에 가까운 가짜분포를 생성하는 것이고 판별모델(D)의 목적은 표본이 가짜분포에 속하는지 진짜분포에 속하는지를 결정하는 것입니다. 이 2가지 모델을 포함한 GAN의 궁극적인 목적은 "실제 데이터의 분포"에 가까운 데이터를 생성하는 것이며, 따라서 판별기의 확률값이 0.5에 가까울 수록 판별기가 가짜 샘플과 실제 샘플을 구별할 수 없는 최적 솔루션으로 판단하게 됩니다. 제안자 Ian Goodfellow은 논문에서 다음과 같이 '경찰과 위조지폐범'을 예시로 들어 모델의 개념을 설명하고 있습니다. (<font style='color:red'>****사진대체예정!!******</font>)

<img src='../pic3/GAN-ch1img01.png'>

생성 모델은 진짜 지폐와 비슷한 가짜 지폐를 만들어 경찰을 속이려 하는 위조지폐범 팀과 같고, 반대로 판별모델은 위조지폐범이 만들어낸 가짜 화폐를 탐지하려는 경찰팀과 유사한데 이러한 경쟁이 계속됨에 따라 위조지폐범 팀은 경찰을 속이지 못한 데이터를, 경찰팀은 위조지폐범에게 속은 데이터를 각각 입력받아 적대적으로 학습하게 되는 것입니다. 이 게임에서의 경쟁은 경쟁은 위조지폐가 진짜 지폐와 구별되지 않을 때까지, 즉 주어진 표본이 실제 표본이 될 확률이 0.5에 가까운 값을 가질 때까지 계속됩니다.

## 1.2. GAN 모델

### 1.2.1. 모델 구조

GAN의 아키텍처를 도식화하면 다음과 같습니다.

<img src='../pic3/GAN-ch1img02.png'>

        그림 1-1 GAN의 아키텍처 (출처: ...)  
        e.g.) Zou et al. 2019. Object Detection in 20 Years: A Survey

생성기(G)는 실제 데이터와 비슷한 데이터를 만들어내도록 학습되며, 판별기(D)는 실제 데이터와 G가 생성한 가짜 데이터를 구별하도록 학습됩니다. GAN의 목적함수는 다음과 같은데, 이는 게임이론 타입의 목적함수로 G와 D 2명의 플레이어가 싸우면서 서로 균형점(nash equilibrium)을 찾아가도록 하는 방식입니다. 

<img src='../pic3/GAN-ch1img03.png'>

        그림 1-2 GAN Value Function (출처: ...)  
        e.g.) Zou et al. 2019. Object Detection in 20 Years: A Survey

여기서 V(D,G)의 값은 확률값으로 도출되는데, 이 수식을 각각 D와 G의 관점에서 살펴보면 다음과 같습니다. <br>

먼저 D의 관점에서 실제 데이터(x)를 입력하면 D(x)가 커지면서 log값이 커지면서 높은 확률값이 나오도록 하고, 가짜 데이터(G(z))를 입력하면 log값이 작아짐에 따라 낮은 확률값이 나오도록 학습됩니다. 다시 말해 D는 실제 데이터와 G가 만든 가짜 데이터를 잘 구분하도록 조금씩 업데이트되는 것입니다. <br>

G에서는 랜덤한 노이즈를 멀티레이어 퍼셉트론에 통과시켜 샘플들을 생성하며 이 가짜 데이터 G(z)를 (??Zero-Mean Gaussian 노이즈 z를 받아 생성된 가짜 데이터를??) D에 input으로 넣었을 때 실제 데이터처럼 확률이 높게 나오도록 학습됩니다. 즉 D(G(z))값을 낮추 전체 확률 값이 낮아지도록 하는 것이며 이는 다시 말해 G가 'D가 잘 구분하지 못하는' 데이터를 생성하도록 조금씩 업데이트되는 것입니다.  <br>

실제 학습을 진행할 때는 G와 D 두 네트워크를 동시에 학습시키지 않고 하나의 네트워크를 고정한 상태에서 다른 한 네트워크를 업데이트하는 방식으로 따로따로 업데이트합니다.

(출처 : paper_00 : Ian J.Goodfellow 논문 & 블로그들 >>> 쉽게쉽게)

### 1.2.2. CGAN

설명) paper_03으로부터.

사례) MNIST from paper_03 간단하게.

### 1.2.3. 평가메트릭 (Evaluation Metrics)

 - 개요
 - 6개 정도 주요한 것들 : FID, IS, Geometry Score, Tournament Based Method, Discriminator Rejection Sampling, Precision and Recall
 - FID, IS에 대해 자세히 설명.

#### Frechet Inception Distance (FID)

(보충) paper_01로부터 & 블로그들  
엔비디아 논문 >> fid로 주로...

(https://analyticsindiamag.com/top-6-metrics-to-monitor-the-performance-of-gans/)  

(...) Frechet Inception Distance는 생성되는 이미지의 퀄리티 일관성을 유지하기 위해 이용되는 메트릭이다. FID가 낮을 수록 이미지의 퀄리티가 더 좋아지며, 이는 다시 말해 실제 이미지와 생성된 이미지의 유사도가 높아지는 것을 말한다. FID는 생성된 샘플들의 통계와 실제 샘플들의 통계를 비교한다.

#### Inception Score (IS)

(보충) paper_01로부터 & 블로그들

**<font style='color:red'>+a</font>**  
**Annealed Importance Sampling**

(https://analyticsindiamag.com/top-6-metrics-to-monitor-the-performance-of-gans/)  

(...)  일반적으로 모델을 비교할 때 샘플링하여 비교하는 것은 매우 번거롭고 그릇될 여지를 가지고 있는 방식입니다. 하지만 Annealed Importance Sampling에서는 decoder-based 모델에 대해서 log-likelihood가 측정되고, 양방향 몬테카를로 알고리즘을 통해 정확도가 검증됩니다.

## 1.3. 적용 사례 (Use Case)

GAN은 이미지 생성을 중심으로 다양하게 응용되어 왔는데 예시를 살펴보면 다음과 같습니다.

### 1.4.1. 가짜이미지 생성 / NVIDIA 


<img src='../pic3/GAN-ch1img98.png'>

출처 : 논문07

2017년 글로벌 GPU 설계 회사 NVIDIA에서 공개한 '실존하지 않는 사람들의 이미지'가 GAN의 대표적인 적용 사례입니다. 당시 NVIDIA는 기존의 GAN 결과들보다 이미지 품질, 안정성, 다양성 등을 향상시킨 새로운 훈련 방법론을 제시하였고 그 결과 생성된 이미지는 위와 같이 사람의 눈으로는 실존 인물인지 가상 인물인지 판별하기 어려운 수준으로 나타났습니다.  

NVIDIA가 제시한 GAN의 새로운 훈련 방법론의 핵심은 Generator와 Discriminator를 둘 다 낮은 <font style='color:red'>'결과값'</font>에서 시작하여 훈련이 진행되면서 모델이 아주 서서히 <font style='color:red'>'학습'</font>하도록 새로운 레이어를 쌓아가며 점진적으로 성장시키는 것입니다. NVIDIA는 이 방식을 통해 기존보다 높은 IS Score(GAN의 평가 메트릭)를 달성했으며 나아가 이미지 퀄리티와 다양성을 모두 고려한 새로운 평가 메트릭을 제안하고 있습니다.

또한, 논문에서 NVIDIA는 사람뿐 아니라 침실, 화분, 소파, 버스 등의 사물도 실제와 같은 이미지로 만들어낼 수 있음을 보여주고 있으며 이는 저해상도의 사진을 고해상도로 만드는 등 손상된 이미지를 복원할 때에도 활용될 수 있습니다.

### 1.4.2. 가짜 오바마 연설 영상 / University of Washington

<img src='../pic3/GAN-ch1img99.png'>

출처 : 논문05

2017년 워싱턴대학교(University of Washington)에서는 GAN을 이용하여 버락 오바마(Barack Obama) 전 미국 대통령의 가짜 연설 영상을 만들어 발표했습니다. 이 영상은 오바마 전 대통령의 과거 연설 영상들로부터 음성을 따고, 이 음성에 맞는 입모양을 만들어 합성한 것으로 가짜입니다. 논문에서 저자는 먼저 오디오 인풋을 시간에 따라 달라지는 입모양으로 변환한 후 진짜같은 입모양을 생성하고, 이를 대상(타겟) 비디오의 입모양 부분에 삽입하여 생성했다고 밝혔습니다. 이렇게 최종 합성 전 입모양 시퀀스와 대상 비디오를 일치시키고 타이밍을 다시 맞추어 머리 움직임과 인풋 스피치가 자연스럽게 나타날 수 있도록 한 것입니다. 영상은 <b><a href='https://www.youtube.com/watch?v=AmUC4m6w1wo&feature=youtu.be'>링크</a></b>에서 시청할 수 있습니다.

### 1.4.3. Eye In-Painting / Facebook

**<font style='color:red'>사진 택일?</font>**

<img src='../pic3/GAN-ch1img97.png'>

<img src='../pic3/GAN-ch1img95.png'>

출처 : paper 09

2017년 12월 Facebook은 ExGAN 기술을 개발하여 사람이 눈을 감은 사진에서 눈을 뜬 사람의 모습의 사진으로 바꾸는 과정을 공개했습니다. Real Eye Openerf라는 명칭으로 불리는 이 작업은 GAN을 통해 진짜같은 가짜 눈을 만들어 눈을 감은 사진에 합성시킨 이미지입니다. 특정 장소나 다시 찍을 수 없는 사진에서 눈을 감고 있는 모습을 보정할 때에 활용할 수 있는 것입니다.

<h4 style='color:red'>+a.</h4>

 - MIT 텍스트 생성(풍경사진 & 짧은 시)  
 - 패션 테크 스타트업 VUE : 고객 개개인의 의상 이미지 제공

## 1.4. 한계점

GAN은 기술적으로 고해상도 이미지를 생성할 수 없다는 점과 학습이 불안정하다는 점을 한계점으로 가지고 있습니다. 이러한 한계점들을 극복하고 다양하게 응용되면서 처음 Ian Goodfellow에 의해 제안된 Vanilla GAN을 시작으로 DCGAN, SRGAN, CycleGAN 등의 GAN 모델이 개발되어 왔습니다.  

사용성 측면에서는 위의 예시들에서처럼 진짜같은 가짜를 생성하는 것이 활용도가 높은 반면 그만큼의 악용 가능성도 존재합니다. 진짜와 가짜를 구별하기 힘들다는 점을 이용한 딥페이크 기술로 만든 포르노 영상이 대표적인 예시인데, 유명인사들의 이미지를 포르노와 합성하여 배포하는 것입니다. 더하여 GAN을 이용하면 이러한 문제가 되는 데이터들을 빠르게 많이 만들어낼 수 있기에 디지털 성범죄 등 윤리적인 이슈도 수반합니다.

이 외에도, GAN을 통해 생성된 미디어의 지식재산권 이슈, 가짜 이미지를 이용한 사기 등 여러가지 법적, 윤리적인 범주의 이슈가 존재하는데 이는 GAN의 기술적 발전에 따른 제도적 대안이 수반되어야 함을 시사합니다.